# Dimension Reduction on Gated Populations

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import joypy
import pacmap
import fitsne
import umap
import flowkit as fk

%matplotlib inline

In [ ]:
def make_plots(data, sample_ids, titles, marker, k=10000, seed=123):
    """Reduce scatter and fluroescent channel data to 2D for visualization."""

    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)

    umap_reducer = umap.UMAP()
    pacmap_reducer = pacmap.PaCMAP()

    X_fitsne = fitsne.FItSNE(data_scaled)
    X_umap = umap_reducer.fit_transform(data_scaled)
    X_pacmap = pacmap_reducer.fit_transform(data_scaled)
    
    min_max_scaler = MinMaxScaler()
    X_reduceds = [
        min_max_scaler.fit_transform(tmp_x) for tmp_x in [X_fitsne, X_umap, X_pacmap]
    ]
    
    n = len(X_reduceds)
    fig, axes = plt.subplots(n, 3, figsize=(n*3, 9))

    for i, (X_reduced, title) in enumerate(zip(X_reduceds, titles)):
        for j in range(3):
            z = X[marker].iloc[(j*k):(j+1)*k]
            ax = axes[j, i]
            ax.scatter(
                X_reduced[(j*k):(j+1)*k, 0], 
                X_reduced[(j*k):(j+1)*k, 1], 
                s=1, 
                c=z, 
                cmap='jet'
            )
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_xlim([-0.1,1.1])
            ax.set_ylim([-0.1,1.1])
            
            if j==0:
                ax.set_title(title, fontsize=14)
            if i==0:
                ax.set_ylabel('-'.join(sample_ids[j].split('_')[3:5]), fontsize=14)
    
    plt.tight_layout()

In [ ]:
base_dir = "../../../data/8_color_data_set"
sample_path = os.path.join(base_dir, "fcs_files")
wsp_path = os.path.join(base_dir, "8_color_ICS.wsp")

In [ ]:
session = fk.Session(sample_path)
session.import_flowjo_workspace(wsp_path)

In [ ]:
sample_groups = session.get_sample_groups()
sample_groups

In [ ]:
sample_group = sample_groups[-1]

In [ ]:
print(session.get_gate_hierarchy(sample_group, output='ascii'))

In [ ]:
sample_ids = sorted(session.get_group_sample_ids(sample_group))
sample_ids

In [ ]:
session.analyze_samples(sample_group)

## Compare marker distributions between CD4+ and CD8+

In [ ]:
k = 10_000

dfs_cd4 = session.get_wsp_gated_events(sample_group, gate_name='CD4+')
dfs_cd8 = session.get_wsp_gated_events(sample_group, gate_name='CD8+')

X_cd4 = pd.melt(pd.concat([df.iloc[:, 2:-1].sample(k) for df in dfs_cd4]))
X_cd8 = pd.melt(pd.concat([df.iloc[:, 2:-1].sample(k) for df in dfs_cd8]))

X = pd.concat([X_cd4, X_cd8], axis=1)
X = X.iloc[:, [0,1,3]]
X.columns = ['marker', 'CD4+', 'CD8+']

In [ ]:
X

In [ ]:
fig, ax = joypy.joyplot(
    X.groupby('marker', sort=False), 
    column=['CD4+', 'CD8+'],
    color=['red', 'blue'],
    legend=True,
    alpha=0.5,
    linewidth=0.5,
    ylim='own',
    figsize=(10, 6)
)

## Visualize dimension reduction schemes for events in Singlet gate

In [ ]:
k = 10_000

dfs = session.get_wsp_gated_events(sample_group, gate_name='Singlets')
X = pd.concat([df.iloc[:, 2:-1].sample(k) for df in dfs])

marker = 'CD4 PE-Cy7 FLR-A'
titles = ['FIt-SNE', 'UMAP', 'PaCMAP']
make_plots(X, sample_ids, titles, marker, k)

## Visualize dimension reduction schemes for events in CD3+ gate

In [ ]:
k = 10_000

dfs = session.get_wsp_gated_events(sample_group, gate_name='CD3+')
X = pd.concat([df.iloc[:, 2:-1].sample(k) for df in dfs])

marker = 'CD4 PE-Cy7 FLR-A'
titles = ['FIt-SNE', 'UMAP', 'PaCMAP']
make_plots(X, sample_ids, titles, marker, k)